# Задача

Аналитику необходимо получить следующую информацию: в каких ресторанах
KFC он может позавтракать (имеется в виду именно завтрак KFC) в Новосибирске
в 8:45.

Мы понимаем, что в дальнейшем могут появится новые задачи по этой теме и
решаем собрать данные по ресторанам KFC в БД, а аналитику отдать SQL-запрос
для получения нужной ему информации.

## Таким образом, задача заключается в следующем:
- скачать с сайта KFC информацию о ресторанах
- сложить эту информацию в базу данных SQLite
- написать запрос для получения информации, которая необходима аналитику

## Что ожидаем получить на выходе:
- работающий код решения на Python или R
- файл с базой данных SQLite с информацией о ресторанах KFC
- соответствующий задаче SQL-запрос к этой базе данных

## Примечания:
- для решения задачи не нужно парсить страницы сайта
- если не получается решить задачу без парсинга, можно получить подсказку, откуда именно нужно скачать данные
- не обязательно складывать в базу всю информацию, которая есть на сайте, достаточно выбрать только основные поля, необходимые для выполнения задания (+координаты и адрес)

## Загрузим данные о ресторанах в базу данных sqlite:

In [1]:
import requests

import sqlite3 as sl

import pandas as pd

# Ссылка с методом для получения информации:

url = 'https://api.kfc.com/api/store/v2/store.get_restaurants'

# Загрузим данные о ресторанах:

response = requests.get(url)

data = response.json()

# Определим количество ресторанов:

restorans_count = len(data['searchResults'])

# Создадим базу данных и связь с ней:

conn = sl.connect("kfc.db")

# Создадим объект класса cursor для взаимодействия с базой данных:

cursor = conn.cursor()

# Удалим неактуальную таблицу, если такая есть:

cursor.execute("""DROP TABLE IF EXISTS restaurants""")

# Создадим таблицу для записи данных:

cursor.execute("""CREATE TABLE restaurants
                  (city TEXT, address TEXT, latitude FLOAT,
                   longitude FLOAT, start_time TEXT, end_time TEXT, 
                   breakfast_start_time TEXT, breakfast_end_time TEXT)
               """)

# Произведём запись полученых с сайта данных в sqlite: 

for i in range(restorans_count):
    
    city = data['searchResults'][i]['storePublic']['contacts']['city']['ru']
    
    try:
    
        address = data['searchResults'][i]['storePublic']['contacts']['streetAddress']['ru']
        
    except LookupError:
        
        address = ''
    
    latitude = data['searchResults'][i]['storePublic']['contacts']['coordinates']['geometry']['coordinates'][0]
    
    longitude = data['searchResults'][i]['storePublic']['contacts']['coordinates']['geometry']['coordinates'][1]
    
    start_time = data['searchResults'][i]['storePublic']['openingHours']['regular']['startTimeLocal']
    
    end_time = data['searchResults'][i]['storePublic']['openingHours']['regular']['endTimeLocal']
    
    breakfast_start_time = data['searchResults'][i]['storePublic']['menues'][0]['availability']['regular']['startTimeLocal']
    
    breakfast_end_time = data['searchResults'][i]['storePublic']['menues'][0]['availability']['regular']['endTimeLocal']
    
    # Добавим строки, которые имеют адрес и непустое значение параметра 'start_time':
    
    if address != '':
        
        if i > 3:
    
            cursor.execute("""INSERT INTO restaurants VALUES (?, ?, ?, ?, ?, ?, ?, ?)""",
                           (city, address, latitude,
                            longitude, start_time, end_time, 
                            breakfast_start_time, breakfast_end_time))

restaurants_df = pd.read_sql('select * from restaurants', conn)

# Сохраним данные:

conn.commit()

# Закроем соединение:
    
conn.close()

## Просмотрим содержимое базы данных:

In [151]:
restaurants_df

,city,address,latitude,longitude,start_time,end_time,breakfast_start_time,breakfast_end_time
0,Санкт-Петербург,"191025, Санкт-Петербург, Маяковского, 1/96",59.932126,30.354398,08:00:00,07:00:00,None,None
1,Санкт-Петербург,"194356, Санкт-Петербург, Выборгское шоссе, 3к1",60.037834,30.320718,08:00:00,23:00:00,09:00:00,11:00:00
2,Москва и МО,"125445, Москва, Правобережная, 1",55.880997,37.449501,10:00:00,20:45:00,None,None
3,Москва и МО,"142704, Москва, Калужское шоссе, 21-й километр",55.604435,37.490441,10:00:00,21:45:00,09:00:00,11:00:00
4,Санкт-Петербург,"195112, Санкт-Петербург, Заневский проспект, 67к2",59.932910,30.437677,10:00:00,21:45:00,10:00:00,11:00:00
...,...,...,...,...,...,...,...,...
1030,Мегион,"628680, Мегион, Нефтяников, 4",61.036363,76.109205,09:00:00,23:00:00,09:00:00,11:00:00
1031,Новокуйбышевск,"446204, Новокуйбышевск, проспект Победы, д. 1Ж",53.101751,49.958486,10:00:00,21:30:00,09:00:00,11:00:00
1032,Междуреченск,"652888, Междуреченск, проспект Шахтёров, дом 15а",53.695134,88.048748,10:00:00,23:00:00,None,None
1033,Челябинск,"454080, Челябинск, ул. Труда, 203",55.171051,61.355976,10:00:00,21:45:00,10:00:00,11:00:00


## Узнаем подходящие адреса ресторанов:

In [32]:
requare_time = '08:45:00'

requare_city = 'Новосибирск'

conn = sl.connect("kfc.db")

cursor = conn.cursor()

cursor.execute(""" SELECT * FROM restaurants WHERE city IN(?) 
                AND time(breakfast_start_time) <= time(?) 
                AND time(breakfast_end_time) >= time(?) """, 
               (requare_city, requare_time, requare_time,))

print('\033[1m' + 'Подходящие адреса ресторанов:' + '\033[0m')

count = 0
               
while True:
    
    row = cursor.fetchone()
        
    if row == None:
        
        break
          
    print('\033[1m' + 'Адрес: ' + '\033[0m' + str(row[1])[8:] + '; ' + 
            '\033[1m' + 'Координаты: ' + '\033[0m' + str(row[2]) + '; ' + str(row[3]) + '; ' + 
            '\033[1m' + 'Начало завтрака: ' + '\033[0m' + str(row[6])[:5] + '; ' + 
            '\033[1m' + 'Окончание завтрака:' + '\033[0m' + str(row[7])[:5])
        
    count += 1

if count == 0:
    
    print('К сожалению нет ничего подходящего..')

conn.close()

Подходящие адреса ресторанов:
Адрес: Новосибирск, Красный проспект, 29; Координаты: 55.031273; 82.919411; Начало завтрака: 08:00; Окончание завтрака:11:00
Адрес: Новосибирск, площадь Карла Маркса, 2; Координаты: 54.983633; 82.892683; Начало завтрака: 08:00; Окончание завтрака:11:00
Адрес: Новосибирск, проспект Карла Маркса, 24; Координаты: 54.989713; 82.908052; Начало завтрака: 08:00; Окончание завтрака:11:00
Адрес: Новосибирск, Дуси Ковальчук, 179/4; Координаты: 55.060571; 82.914217; Начало завтрака: 07:00; Окончание завтрака:11:00
Адрес: Новосибирск, Троллейная, 130; Координаты: 54.964728; 82.853306; Начало завтрака: 08:00; Окончание завтрака:11:00
Адрес: Новосибирск, проезд Энергетиков, 9; Координаты: 55.009649; 82.877532; Начало завтрака: 06:00; Окончание завтрака:11:00
Адрес: Новосибирск, Курчатова, 1; Координаты: 55.104311; 82.960692; Начало завтрака: 08:00; Окончание завтрака:11:00
Адрес: Новосибирск, Станционная, 32; Координаты: 54.997494; 82.851273; Начало завтрака: 08:00; Око